In [1]:
import geopandas as gpd
import numpy as np
from scipy.spatial import Voronoi
from shapely.geometry import Polygon
from shapely.geometry import box
from shapely.geometry import MultiPoint
from shapely.ops import triangulate

Read in BFE shapefile. This was manually created for the MRE(min. reproducible example). 'Name' would be a BFE ID, where 'Notes' could hold elevation values in meters.

In [58]:
bf = gpd.read_file('BFE.shp')
bf.rename(columns={'Notes': 'Elevation'}, inplace=True)
bf = bf[['Name', 'Elevation', 'geometry']].to_file('BFEs.shp')

D:\mini-forge\envs\geopandas\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [3]:
bf

,Name,Notes,geometry
0,BF1,10,"LINESTRING Z (-96.79929 32.81435 0.00000, -96...."
1,BF2,11,"LINESTRING Z (-96.79979 32.81355 0.00000, -96...."
2,BF3,8,"LINESTRING Z (-96.80041 32.81350 0.00000, -96...."
3,BF4,5,"LINESTRING Z (-96.80109 32.81304 0.00000, -96...."
4,BF5,2,"LINESTRING Z (-96.80113 32.81192 0.00000, -96...."


In [4]:
bf.explore()

Extract x,y coordinates from the BFE shapefile and organize them into a numpy array.

In [5]:
pts = []
for p in bf['geometry']:
    x1 = p.xy[0][0]
    y1 = p.xy[1][0]
    x2 = p.xy[0][1]
    y2 = p.xy[1][1]
    pts.append([[x1,y1],[x2,y2]])

In [6]:
pts_coords = []
idx = 0
while idx < 2:
    for p in pts:
        pts_coords.append(p[idx])
    idx += 1

In [7]:
# resulting array
pts_coords

[[-96.79928865999995, 32.81434883000003],
 [-96.79979212799998, 32.813547010000036],
 [-96.80040747699996, 32.81350039300003],
 [-96.80108809199999, 32.81304354200006],
 [-96.80112538599997, 32.81191540100008],
 [-96.79837495799995, 32.813770774000034],
 [-96.79877586799995, 32.81306218900005],
 [-96.80002521499995, 32.812502780000045],
 [-96.80052868299998, 32.812353604000066],
 [-96.80043544799997, 32.81187810700004]]

Triangulate points form array and covert to geodataframe.

In [8]:
tri_p = MultiPoint(pts_coords)
triangles = triangulate(tri_p)
poly = gpd.GeoDataFrame(geometry=triangles, crs='EPSG:4326')

In [9]:
poly.explore()

Triangles that are not within the Original flood surface polygons will need to be removed. Add in the overlay Flood Surface Polygon and clip to the extent. 

In [10]:
fsp = gpd.read_file('FSP.shp')
fsp.explore()

In [11]:
tris = poly.clip(fsp)

In [12]:
tris.explore()

In [47]:
bf_pts = gpd.GeoDataFrame(geometry=gpd.GeoSeries(map(Point, pts_coords), crs='EPSG:4326'))
bf_pts = gpd.sjoin(bf_pts, bf, how='left')

In [48]:
bf_pts.explore()

In [52]:
tris

,geometry
9,"POLYGON Z ((-96.80044 32.81188 0.00000, -96.80..."
8,MULTILINESTRING Z ((-96.80003 32.81250 0.00000...
0,"POLYGON Z ((-96.80109 32.81304 0.00000, -96.80..."
1,"POLYGON Z ((-96.80109 32.81304 0.00000, -96.80..."
2,"POLYGON Z ((-96.80041 32.81350 0.00000, -96.80..."
3,"POLYGON Z ((-96.80041 32.81350 0.00000, -96.79..."
7,GEOMETRYCOLLECTION Z (LINESTRING Z (-96.79878 ...
10,"POLYGON Z ((-96.80003 32.81250 0.00000, -96.79..."
5,"POLYGON Z ((-96.79929 32.81435 0.00000, -96.79..."
6,"POLYGON Z ((-96.79929 32.81435 0.00000, -96.79..."


In [55]:
bf_pts.rename(columns={'Notes': 'Elevation'}, inplace=True)
bf_pts = bf_pts[['Name', 'Elevation', 'geometry']].to_file('bf_pts.shp')

D:\mini-forge\envs\geopandas\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
